<a href="https://colab.research.google.com/github/vasu-rguktn/Team_X/blob/main/Model_Building_with_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas numpy scikit-learn xgboost pyarrow

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from xgboost import XGBClassifier

In [3]:
!unzip archive.zip

Archive:  archive.zip
  inflating: building_metadata.feather  
  inflating: sample_submission.feather  
  inflating: test.feather            
  inflating: train.feather           
  inflating: weather_test.feather    
  inflating: weather_train.feather   


In [5]:
df=pd.read_feather("train.feather")
df = df.sample(n=500_000, random_state=42)


In [6]:
df.columns



Index(['building_id', 'meter', 'timestamp', 'meter_reading'], dtype='object')

In [7]:
df.fillna(df.median(numeric_only=True), inplace=True)


In [8]:
threshold = df['meter_reading'].median()
df['target'] = (df['meter_reading'] > threshold).astype(int)

df.drop(columns=['meter_reading'], inplace=True)


In [9]:
x=df.drop(columns=['target'])
y=df['target']

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

In [13]:
for df_ in [x_train, x_test]:
    df_['hour'] = df_['timestamp'].dt.hour
    df_['day'] = df_['timestamp'].dt.day
    df_['month'] = df_['timestamp'].dt.month
    df_.drop(columns=['timestamp'], inplace=True)


In [14]:
model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=0.1,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=-1,
              num_parallel_tree=None, ...)

In [15]:
from xgboost import XGBClassifier
model = XGBClassifier(
    n_estimators=120,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=5,
    gamma=0.1,
    reg_lambda=1.0,
    eval_metric='logloss',
    tree_method='hist',
    random_state=42,
    n_jobs=-1
)

model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=0.1,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=-1,
              num_parallel_tree=None, ...)

In [17]:
import numpy as np
from sklearn.metrics import mean_squared_log_error

y_pred = model.predict(x_test)
y_pred = np.maximum(0, y_pred)

rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print("RMSLE:", rmsle)


RMSLE: 0.37070739886199083
